In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
PROJ_HOME = '/Users/skeem396/Projects/'
sys.path.append(PROJ_HOME)

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import top_k_top_p_filtering
import torch
from torch.nn import functional as F

In [15]:
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')

Downloading:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [16]:
generator("FreeWheel is", do_sample=True, min_length=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'FreeWheel is the original, largest, and most popular choice for bike rentals in Houston. Our company has been providing quality bike rentals and repair services for over three decades. In 2010, Over the Road Bike provided a new name and a new beginning,'}]

In [17]:
generator("Comcast is", do_sample=True, min_length=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Comcast is expanding its cable bundle with new digital tiers for new subscribers, and will roll out more deals as part of an expansion of its current lineup in the coming months.\n\nThe company will offer customers in the US starting today for the first'}]

In [18]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")

In [19]:
test_text = """How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black""" 

In [20]:
tokenizer(test_text)

{'input_ids': [2437, 867, 517, 15102, 761, 284, 307, 2923, 287, 1502, 329, 262, 1705, 12527, 11, 2592, 8661, 3392, 11, 284, 892, 326, 356, 447, 247, 260, 12733, 286, 257, 1621, 30, 2619], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
def generate_n_next(phrase, length=10):
    for i in range(length):
        inputs = tokenizer.encode(phrase, return_tensors='pt')

        # Get logits from last layer
        last_layer_logits = model(inputs).logits[:, -1, :]
        top_logits = top_k_top_p_filtering(last_layer_logits, top_k=100, top_p=1.0)

        # Softmax the logits into probabilities
        probabilities = F.softmax(top_logits, dim=-1)

        # Generate next token
        generated_next_token = torch.multinomial(probabilities, num_samples=1)
        generated = torch.cat([inputs, generated_next_token], dim=-1)

        tok_id = generated.tolist()[0]
        result_string = tokenizer.decode(tok_id)
        print(result_string)
        phrase = result_string
    return phrase

GPT-Neo was trained on the Pile, a dataset known to contain profanity, lewd, and otherwise abrasive language. Depending on your usecase GPT-Neo may produce socially unacceptable text. 

In [22]:
generate_n_next(test_text, length=10)

How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people are
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people are already
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people are already constantly
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people are already constantly being
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people are already constantly being tol

'How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Black people are already constantly being told not to call out'

In [23]:
test_text2 = """How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian""" 

In [24]:
generate_n_next(test_text2, length=10)

How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans
How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans die
How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans die at
How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans die at a
How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans die at a higher
How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans die at a higher rate
How many more Asians need

'How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story? Asian Americans die at a higher rate than any other group'

In [25]:
test_text3 = """How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story""" 
generate_n_next(test_text3, length=10)

How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like this
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like this,
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like this, which
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like this, which only
How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like this, which only gets
How many more blacks need to be killed in order for the news outlets, especially mainstream 

'How many more blacks need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story like this, which only gets worse after it�'

In [26]:
test_text4 = """How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story"""
generate_n_next(test_text4, length=10)

How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?
How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?

How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?


How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?



How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?




How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?





How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?




This
How many more Asi

'How many more Asians need to be killed in order for the news outlets, especially mainstream ones, to think that we’re worthy of a story?\n\n\n\n\nThis piece first appeared'